In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Mon Jan  9 16:29:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   33C    P8    32W / 280W |    905MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [5]:
train_x = dd.read_csv('../input/higgs_small_roc/xgtrain_aug_5.csv')
test_x = dd.read_csv('../input/higgs_small_roc/xgval_aug.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2023-01-09 16:30:11,858 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-09 16:30:11,859 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-01-09 16:30:11,863 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-09 16:30:11,863 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [12]:
study = optuna.create_study(direction='maximize')

[I 2023-01-09 16:30:22,623] A new study created in memory with name: no-name-92d1e072-ed8b-4245-b581-8ca5de66bdbc


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:30:24] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0
[I 2023-01-09 16:30:29,897] Trial 0 finished with value: 0.7906961987379522 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.6050311712241306, 'alpha': 3.6704860933316574, 'colsample_bytree': 0.3731494171901087, 'subsample': 0.9372368597726295, 'learning_rate': 0.006007494687533758, 'max_depth': 5, 'min_child_weight': 37, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7906961987379522.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:30:29] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0
[I 2023-01-09 16:30:33,132] Trial 1 finished with value: 0.789009389

CPU times: user 649 ms, sys: 143 ms, total: 792 ms
Wall time: 8.92 s


In [14]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:30:42] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0
[I 2023-01-09 16:31:07,037] Trial 2 finished with value: 0.8109703877038656 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.6190144360307883, 'alpha': 0.015346075851411402, 'colsample_bytree': 0.6856321141454566, 'subsample': 0.5866610922148862, 'learning_rate': 0.022823021959782263, 'max_depth': 15, 'min_child_weight': 60, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.8109703877038656.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[16:31:07] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0
[I 2023-01-09 16:31:34,022] Trial 3 finished with value: 0.790768

CPU times: user 1min 23s, sys: 38.8 s, total: 2min 2s
Wall time: 46min 58s


In [15]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 2.5032983959155435,
 'alpha': 0.0013590474814060799,
 'colsample_bytree': 0.8912585832206631,
 'subsample': 0.5428660940893997,
 'learning_rate': 0.01700089087330609,
 'max_depth': 17,
 'min_child_weight': 31,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/higgs_small_roc/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
params = study.best_trial.params

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

roc_auc_score(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[21:00:23] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0


0.8065767574394525

In [20]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()

    scores.append(roc_auc_score(real_test_y, predictions))

0


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[21:01:09] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0


1


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[21:01:56] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0


2


...

13


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[21:11:18] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0


14


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[21:12:06] task [xgboost.dask-0]:tcp://127.0.0.1:42715 got new rank 0


CPU times: user 22.5 s, sys: 7.24 s, total: 29.7 s
Wall time: 11min 42s


In [21]:
scores

[0.8064256477406464,
 0.8060767440034209,
 0.8062614098649692,
 0.8071505464290549,
 0.8064136466250645,
 0.8061058440997689,
 0.8064499421728855,
 0.8061004540335096,
 0.8067507318723801,
 0.8065671461112341,
 0.8065574878221288,
 0.8064025847272238,
 0.8063274264364226,
 0.8068129915728688,
 0.8068820188589704]

In [22]:
import numpy as np
np.mean(scores)

0.8064856414913699